### 데이터 수집 (1) 한 페이지 수집

#### 다나와 홈페이지 접속

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome('C:/Users/NK/Desktop/programming/python/chromedriver.exe')
url = 'http://search.danawa.com/dsearch.php?k1=무선청소기&module=goods&act=dispMain#'
driver.get(url)
time.sleep(3)

#### 상품 정보 가져오기

In [2]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [3]:
# 1페이지에 대한 무선청소기 정보 가져오기

prod_items = soup.select('li.prod_item')
len(prod_items)

49

In [4]:
# 1페이지에 대한 무선청소기 정보 가져오기

prod_items = soup.select('ul.product_list > li.prod_item')
len(prod_items)

49

In [5]:
# 1페이지에 대한 무선청소기 정보 가져오기

prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
len(prod_items)

40

In [6]:
# 첫번째 상품의 정보 가져오기

## 상품명
title = prod_items[0].select('a.click_log_product_standard_title_')[0].text.strip()

## 스펙
spec = prod_items[0].select('div.spec_list')[0].text.strip()

## 가격
price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')

print(title)
print(spec)
print(price)

LG전자 코드제로 A9S A9700
핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 헤파필터 / 2중터보싸이클론 / 5단계여과 / 인버터모터 / 흡입력: 200W / 충전시간: 4시간 / 사용시간: 2시간 / 배터리: 리튬이온 (2개) / 브러쉬: 바닥, 솔형, 틈새, 침구, 매트리스, 물걸레, 연장관 / 거치대 / 디스플레이표시창 / 배터리잔량표시 / WiFi / 색상:판타지실버 / 소비전력: 590W / 미세먼지인증(SLG) / 3단계물분사조절 / 크기(가로x세로x깊이): 250x1120x260mm
1010320


In [7]:
# 첫페이지의 모든 상품의 정보를 가져오는 반복문
import pandas as pd

prod_data = []

for item in prod_items:
    try:
        title = item.select('a.click_log_product_standard_title_')[0].text.strip()
    except:
        title = ''
    
    try:
        spec = item.select('div.spec_list')[0].text.strip()
    except:
        spec = ''
    
    try:
        price = item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
    except:
        price = ''
        
    prod_data.append([title, spec, price])
    
print(len(prod_data))
pd.DataFrame(prod_data).head()

40


,0,1,2
0,LG전자 코드제로 A9S A9700,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 헤파필터 / 2중터보싸이클론 / 5단...,1010320
1,다이슨 V8 플러피,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 전압: 21.6V / 싸이클론 / 흡...,405230
2,샤오미 드리미 V10,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 전압: 25.2V / 헤파필터 / H...,162980
3,일렉트로룩스 ZB3302AK,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 전압: 14.4V / 4단계여과 / ...,124380
4,원더스리빙 원더스 다이나킹 Z9,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 전압: 28.8V / 헤파필터 / 듀...,299000


In [8]:
# 한 페이지의 검색결과를 수집하는 함수

def get_prod_items(prod_items):
    prod_data = []
    
    prod_data = []

    for item in prod_items:
        try:
            title = item.select('a.click_log_product_standard_title_')[0].text.strip()
        except:
            title = ''
    
        try:
            spec = item.select('div.spec_list')[0].text.strip()
        except:
            spec = ''
    
        try:
            price = item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
            price = int(price)
        except:
            price = ''

        prod_data.append([title, spec, price])
        
    return prod_data

In [9]:
# 함수 테스트

prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
prod_data = get_prod_items(prod_items)
print(len(prod_data))

40


### 데이터 수집 (2) 여러 페이지에 걸친 다나와 검색 페이지

In [10]:
# 검색페이지의 URL을 보면 query / page 만 바뀌는 것을 알 수 있음

# 다나와 검색 url을 만들어주는 함수
def get_search_page_url(keyword, page):
    return f'http://search.danawa.com/dsearch.php?query={keyword}&originalQuery={keyword}&volumeType=allvs&page={page}&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods'

In [11]:
keyword = '무선청소기'
page = '3'
url = get_search_page_url(keyword, page)
print(url)

http://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&volumeType=allvs&page=3&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods


### 주피터 노트북의 진행표시줄 처리

In [12]:
import time
from tqdm import tqdm_notebook

total_page = 10
for page in tqdm_notebook(range(1, total_page+1)):
    # 페이지 로딩 완료되기 위한 시간을 5초로 한다
    time.sleep(5)

C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


### 여러 페이지에 걸친 상품 정보 수집

In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

import pandas as pd
import time

driver = webdriver.Chrome('C:/Users/NK/Desktop/programming/python/chromedriver.exe')
driver.implicitly_wait(3)

keyword = '무선청소기'
total_page = 10
prod_data_total = []
# 진행 정도를 표현하는 tqdm을 적용
for page in tqdm_notebook(range(1, total_page + 1)):

    # 1. 검색페이지 이동
    url = get_search_page_url(keyword, page)
    driver.get(url)
    ## 페이지 로딩 완료까지 5초를 기다림
    time.sleep(5)
    
    # 2. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 3. 상품 정보 추출
    prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
    prod_item_list = get_prod_items(prod_items)
    
    # 4. 추출 데이터 저장
    prod_data_total += prod_item_list
    
len(prod_data_total)

C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


400

In [14]:
# 수집 데이터 확인
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명', '스펙 목록', '가격']
print(data.shape)
data.head()

(400, 3)


,상품명,스펙 목록,가격
0,LG전자 코드제로 A9S A9700,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 헤파필터 / 2중터보싸이클론 / 5단...,1010320
1,다이슨 V8 플러피,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 전압: 21.6V / 싸이클론 / 흡...,405230
2,샤오미 드리미 V10,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 전압: 25.2V / 헤파필터 / H...,162980
3,일렉트로룩스 ZB3302AK,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 전압: 14.4V / 4단계여과 / ...,124380
4,원더스리빙 원더스 다이나킹 Z9,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 전압: 28.8V / 헤파필터 / 듀...,299000


In [15]:
# 수집 데이터 저장
data.to_excel('crawling_data/6_1_danawa_crawling_result.xlsx', index = False)